In [59]:
import pandas as pd
import numpy as np
import random as rnd
import string
import uuid
import matplotlib as matplotlib

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [60]:
#loading datasets
train_df = pd.read_csv('../Datasets/aus_fl.csv')
test_df = pd.read_csv('../Datasets/aus_fl.csv')

In [61]:
#analysing data
print(train_df.columns.values)

#preview the data
train_df.head()

train_df["Departing_Port"].unique()

['Route' 'Departing_Port' 'Arriving_Port' 'Airline' 'Month'
 'Sectors_Scheduled' 'Sectors_Flown' 'Cancellations' 'Departures_On_Time'
 'Arrivals_On_Time' 'Departures_Delayed' 'Arrivals_Delayed' 'Year'
 'Month_Num']


array(['Adelaide', 'Albury', 'Alice Springs', 'All Ports', 'Brisbane',
       'Broome', 'Burnie', 'Cairns', 'Canberra', 'Coffs Harbour',
       'Darwin', 'Devonport', 'Dubbo', 'Gold Coast', 'Hobart',
       'Kalgoorlie', 'Launceston', 'Mackay', 'Melbourne', 'Mildura',
       'Perth', 'Rockhampton', 'Sunshine Coast', 'Sydney', 'Townsville',
       'Wagga Wagga', 'Proserpine', 'Newcastle', 'Ballina', 'Karratha',
       'Hamilton Island', 'Hervey Bay', 'Port Hedland', 'Port Lincoln',
       'Port Macquarie', 'Newman', 'Ayers Rock', 'Gladstone', 'Geraldton',
       'Emerald', 'Mount Isa', 'Bundaberg', 'Moranbah', 'Armidale',
       'Tamworth'], dtype=object)

Aqui podemos observar que el atributo Month nos da información sobre el mes pero no en referencia a un año. Es decir no nos da un número del 1 al 12, que sería lo normal. Sin embargo nos devuelve un numero muy grande que nos puede dar problemas, por lo que probablemente lo eliminaremos más adelante.

Atributos de interés; Route: nos dice la ruta que sigue el vuelo, Departing_Port: Puerto de Salida, Airline: Aerolina con la que se realiza la ruta, Sectors_Scheduled: Vuelos programados, Sectors_Flown: Vuelos despegados, Cancellations: Cancelaciones, Departures_On_Time: Salidas a tiempo, Arrivals_On_Time: Llegadas a tiempo, Year: Año del vuelo, Month_Num: Mes (Este si nos interesa)

A continuación procederemos a la limpieza del dataset para poder hacer un segundo análisis más conciso y detallado.

Ahora procederemos a eliminar dos atributos que no nos sirven de nada que son Arriving and Departure Port.

In [62]:
del train_df['Route']
del train_df['Month']

In [63]:
train_df.head()

,Departing_Port,Arriving_Port,Airline,Sectors_Scheduled,Sectors_Flown,Cancellations,Departures_On_Time,Arrivals_On_Time,Departures_Delayed,Arrivals_Delayed,Year,Month_Num
0,Adelaide,Brisbane,All Airlines,155.0,155,0.0,123.0,120.0,32.0,35.0,2004,1
1,Adelaide,Canberra,All Airlines,75.0,75,0.0,72.0,72.0,3.0,3.0,2004,1
2,Adelaide,Gold Coast,All Airlines,40.0,40,0.0,36.0,35.0,4.0,5.0,2004,1
3,Adelaide,Melbourne,All Airlines,550.0,548,2.0,478.0,487.0,70.0,61.0,2004,1
4,Adelaide,Perth,All Airlines,191.0,191,0.0,169.0,168.0,22.0,23.0,2004,1


In [64]:
for elem in train_df['Arriving_Port'].unique():
    train_df[str(elem)] = train_df['Arriving_Port'] == elem
for elem in train_df['Departing_Port'].unique():
    train_df[str(elem)] = train_df['Departing_Port'] == elem
for elem in train_df['Arriving_Port'].unique():
    train_df[str(elem)] = train_df['Arriving_Port'] == elem
for elem in train_df['Airline'].unique():
    train_df[str(elem)] = train_df['Airline'] == elem 

In [65]:
train_df.head()

,Departing_Port,Arriving_Port,Airline,Sectors_Scheduled,Sectors_Flown,Cancellations,Departures_On_Time,Arrivals_On_Time,Departures_Delayed,Arrivals_Delayed,...,Regional Express,Skywest,Virgin Australia,Jetstar,Macair,Ozjet,MacAir,Tigerair Australia,Virgin Australia - ATR/F100 Operations,Virgin Australia Regional Airlines
0,Adelaide,Brisbane,All Airlines,155.0,155,0.0,123.0,120.0,32.0,35.0,...,False,False,False,False,False,False,False,False,False,False
1,Adelaide,Canberra,All Airlines,75.0,75,0.0,72.0,72.0,3.0,3.0,...,False,False,False,False,False,False,False,False,False,False
2,Adelaide,Gold Coast,All Airlines,40.0,40,0.0,36.0,35.0,4.0,5.0,...,False,False,False,False,False,False,False,False,False,False
3,Adelaide,Melbourne,All Airlines,550.0,548,2.0,478.0,487.0,70.0,61.0,...,False,False,False,False,False,False,False,False,False,False
4,Adelaide,Perth,All Airlines,191.0,191,0.0,169.0,168.0,22.0,23.0,...,False,False,False,False,False,False,False,False,False,False


Ahora tenemos tanto Departing, Arriving Port como Airline transformados en booleanos para que nuestro modelo los pueda tratar. Sin embargo lo que se ha dado como consecuencia es que se han añadido los atributos en forma de columnas. Esto no debe afectar mucho al tratamiento de nuestro dataset ya que es muy largo (mas de 75000 reg)

Eliminaremos columnas inservibles

In [66]:
del train_df["Airline"]
del train_df["Departing_Port"]
del train_df["Arriving_Port"]

In [67]:
train_df.head()

,Airline,Sectors_Scheduled,Sectors_Flown,Cancellations,Departures_On_Time,Arrivals_On_Time,Departures_Delayed,Arrivals_Delayed,Year,Month_Num,...,Regional Express,Skywest,Virgin Australia,Jetstar,Macair,Ozjet,MacAir,Tigerair Australia,Virgin Australia - ATR/F100 Operations,Virgin Australia Regional Airlines
0,All Airlines,155.0,155,0.0,123.0,120.0,32.0,35.0,2004,1,...,False,False,False,False,False,False,False,False,False,False
1,All Airlines,75.0,75,0.0,72.0,72.0,3.0,3.0,2004,1,...,False,False,False,False,False,False,False,False,False,False
2,All Airlines,40.0,40,0.0,36.0,35.0,4.0,5.0,2004,1,...,False,False,False,False,False,False,False,False,False,False
3,All Airlines,550.0,548,2.0,478.0,487.0,70.0,61.0,2004,1,...,False,False,False,False,False,False,False,False,False,False
4,All Airlines,191.0,191,0.0,169.0,168.0,22.0,23.0,2004,1,...,False,False,False,False,False,False,False,False,False,False


In [68]:
train_df=train_df.dropna()

In [69]:
train_df['Sectors_Scheduled'] = pd.to_numeric(train_df['Sectors_Scheduled']).astype(int)
train_df['Cancellations'] = pd.to_numeric(train_df['Cancellations']).astype(int)
train_df['Departures_On_Time'] = pd.to_numeric(train_df['Departures_On_Time']).astype(int)
train_df['Arrivals_On_Time'] = pd.to_numeric(train_df['Arrivals_On_Time']).astype(int)
train_df['Arrivals_Delayed'] = pd.to_numeric(train_df['Arrivals_Delayed']).astype(int)
train_df['Departures_Delayed'] = pd.to_numeric(train_df['Departures_Delayed']).astype(int)

In [70]:
train_df.head()

,Airline,Sectors_Scheduled,Sectors_Flown,Cancellations,Departures_On_Time,Arrivals_On_Time,Departures_Delayed,Arrivals_Delayed,Year,Month_Num,...,Regional Express,Skywest,Virgin Australia,Jetstar,Macair,Ozjet,MacAir,Tigerair Australia,Virgin Australia - ATR/F100 Operations,Virgin Australia Regional Airlines
0,All Airlines,155,155,0,123,120,32,35,2004,1,...,False,False,False,False,False,False,False,False,False,False
1,All Airlines,75,75,0,72,72,3,3,2004,1,...,False,False,False,False,False,False,False,False,False,False
2,All Airlines,40,40,0,36,35,4,5,2004,1,...,False,False,False,False,False,False,False,False,False,False
3,All Airlines,550,548,2,478,487,70,61,2004,1,...,False,False,False,False,False,False,False,False,False,False
4,All Airlines,191,191,0,169,168,22,23,2004,1,...,False,False,False,False,False,False,False,False,False,False


In [71]:
#matplotlib.pyplot.hist(data=train_df, **2)[0:3]

In [72]:
#yearly_counts = surveys_complete.groupby(['Airline', 'Sectors_Flown'])['Sectors_Flown'].count()
#yearly_counts

In [73]:
#train_df['grade'] = pd.to_numeric(data_df['grade'], errors='coerce')
 #                    .fillna(0)
  #                   .astype(int)

In [74]:
X_train = train_df.copy()
Y_train = train_df['Cancellations'].copy()
Y_train = Y_train.dropna(how='any')

In [75]:
X_train=X_train[0:72000] 
Y_train=Y_train[0:72000] 

In [76]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train, Y_train, test_size=0.20, train_size=0.80) 

In [77]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(xtrain,ytrain)

C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [78]:
predictions = logmodel.predict(xtest)
#Evaluation of our model

from sklearn.metrics import classification_report, accuracy_score
#Checking precision, recall, f1-score using classification report

#print(classification_report(ytest,predictions))
print(accuracy_score(ytest,predictions))

0.7118055555555556


In [84]:
submission = pd.DataFrame({
    'Cancelaciones': xtest['Cancellations'],
    'Month': xtest['Month_Num'],
    'Year': xtest['Year'],
})

submission.to_csv('logRg_AusFl.csv', index=False)

In [23]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(xtrain, ytrain)

Y_pred = decision_tree.predict(xtest)

# Any results you write to the current directory are saved as output.

submission = pd.DataFrame({
    'Cancelaciones': xtest['Cancellations'],
    'Month': xtest['Month_Num'],
    'Year': xtest['Year'],
    'Airline': xtest['Airline']
})

submission.to_csv('decTree_AusFl.csv', index=False)

In [29]:
predictionsDTC = decision_tree.predict(xtest)
print(classification_report(ytest,Y_pred))
print(accuracy_score(ytest,predictionsDTC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7880
           1       1.00      1.00      1.00      2332
           2       1.00      1.00      1.00      1147
           3       1.00      1.00      1.00       640
           4       1.00      1.00      1.00       411
           5       1.00      1.00      1.00       300
           6       1.00      1.00      1.00       214
           7       1.00      1.00      1.00       155
           8       1.00      1.00      1.00       144
           9       1.00      1.00      1.00       130
          10       1.00      1.00      1.00        77
          11       1.00      1.00      1.00        72
          12       1.00      1.00      1.00        61
          13       1.00      1.00      1.00        49
          14       1.00      1.00      1.00        57
          15       1.00      1.00      1.00        40
          16       1.00      1.00      1.00        22
          17       1.00    

C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\gtorres\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0

In [25]:
#(data,label)
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier()
clf.fit(xtrain,ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [26]:
#(test_data)
predictions=clf.predict(xtest)

In [28]:
#(test_labels,predictions)
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest,predictions))

0.5627083333333334
